### Antibody escape lineplot
This notebook reads in filtered antibody escape data and generates a lineplot showing summed escape across the different antibodies. 

In [ ]:
import pandas as pd

import altair as alt

import httpimport

_ = alt.data_transformers.disable_max_rows()

In [ ]:
# Import custom altair theme from remote github using httpimport module
def import_theme_new():
    with httpimport.github_repo("bblarsen-sci", "altair_themes", "main"):
        import main_theme

        @alt.theme.register("custom_theme", enable=True)
        def custom_theme():
            return main_theme.main_theme()


import_theme_new()

In [ ]:
antibody_escape_df = pd.read_csv(snakemake.input.escape_df)
antibody_order = snakemake.params.antibody_order

In [ ]:
#antibody_escape_df = pd.read_csv('../../results/filtered_data/antibody_escape/combined/all_antibodies_escape_filtered_sum.csv')
#antibody_order = ["12B2", "2D3", "4H3", "1A9", "1F2", "2B12"]

In [ ]:
# Define x axis properties
x_axis = alt.Axis(
    labelAngle=0,
    title="Site",
    values=[100, 200, 300, 400, 500, 600],
)

# only show positive escape values
antibody_escape_df["sum_escape"] = antibody_escape_df["sum_escape"].clip(lower=0)

# Create line plot with points for each antibody
empty_charts = []
for antibody in antibody_order:
    antibody_escape_chart = (
        alt.Chart(antibody_escape_df.query('antibody == @antibody'))
        .mark_line(
            point=alt.OverlayMarkDef(
                filled=False, fill="white", size=50, strokeWidth=2, color="black"
            ),
            interpolate="monotone",
            strokeWidth=1,
            color="#B8B0AC",
        )
        .encode(
            x=alt.X("site:N", title="Site").axis(x_axis),
            y=alt.Y("sum_escape:Q", title="Escape", axis=alt.Axis(tickCount=3)),
            tooltip=["site", "antibody", "sum_escape"],
        )
        .properties(height=100, width=150, title=antibody)
    )
    empty_charts.append(antibody_escape_chart)

antibody_escape_chart = alt.vconcat(*empty_charts).resolve_scale(y="shared", x="shared")
antibody_escape_chart.display()
#antibody_escape_chart.save('../../results/figures/antibody_escape/antibody_escape_lineplot.svg')

In [ ]:
antibody_escape_chart.save(snakemake.output.antibody_escape_line_plot_svg)
antibody_escape_chart.save(snakemake.output.antibody_escape_line_plot_png, ppi=300)
antibody_escape_chart.save(snakemake.output.antibody_escape_line_plot_html)
